# Hungry Geese - Rust CFFI

This is an attempt to port my [Hungry Geese Go West!](https://www.kaggle.com/jamesmcguigan/hungry-geese-go-west) notebook to Rust using CFFI bindings.

This is inspired by this tutorial: https://bheisler.github.io/post/calling-rust-in-python/

# Install Rust

We use the rustup script, and also create the directory structure that Cargo expects

In [ ]:
!mkdir ./src/
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs > rustup.sh
!bash ./rustup.sh -y

# Cargo File

In [ ]:
%%writefile Cargo.toml
[package]
name    = "hungry-geese-rust-west"
version = "0.1.0"
authors = [ "JamesMcGuigan <james.mcguigan@gmail.com>" ]
edition = "2018"

[lib]
name = "hungry_geese_rust_west"
crate-type = ["dylib"]

# Rust Code

It is tricky to return variable length strings from Rust.
- return `String` type is not supported by FFI
- return `str` type complains that the string is variable length
- we can work around this by simply returning a number, and converting to a string in python

In [ ]:
%%writefile src/lib.rs
// DOCS: https://bheisler.github.io/post/calling-rust-in-python/

#[no_mangle]
pub extern "C" fn go_west() -> i32 {
    return 3; // NORTH, EAST, SOUTH, WEST
}

# Compiling Rust

Kaggle Simulations have an issue with multi-directory submission files. So everything required to run should be copied to the root directory.
- https://www.kaggle.com/c/rock-paper-scissors/discussion/216159

In [ ]:
!source ~/.cargo/env; cargo build
!cp ./target/debug/*.so ./
!rm -rf ./target 

In [ ]:
!find ./

# Python Code

Here we need to statically define out functional imports, and find the correct path for the compiled binary.

The python file itself also needs to be called `main.py`

In [ ]:
%%writefile main.py
# DOCS: https://bheisler.github.io/post/calling-rust-in-python/

# BUGFIX: Kaggle Submission Environment os.getcwd() == "/kaggle/working/"
# https://www.kaggle.com/c/rock-paper-scissors/discussion/216159
import os
import glob
if "simulations" in os.environ.get('KAGGLE_DOCKER_IMAGE', ''):
    os.chdir('/kaggle_simulations/agent/')
    print("os.getcwd() ", os.getcwd())
    print("glob.glob(*)", glob.glob("*"))
    print("os.environ  ", os.environ)
    

from cffi import FFI
ffi = FFI()
ffi.cdef("""
    int go_west();
""")
rust = ffi.dlopen("./libhungry_geese_rust_west.so")



from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col

# obs  {'remainingOverageTime': 60, 'index': 1, 'step': 30, 'geese': [[56], [68], [51, 52], [54]], 'food': [8, 15]}
# conf {'episodeSteps': 200, 'actTimeout': 1, 'runTimeout': 1200, 'columns': 11, 'rows': 7, 'hunger_rate': 40, 'min_food': 2, 'max_length': 99}
def go_west_go_agent(obs, conf):
    # if obs.step >= conf.hunger_rate-2: raise Exception("Don't submit to competition")
    action = rust.go_west();
    return list(Action)[action].name

Test the python file compiles

In [ ]:
%run main.py

# Kaggle Simulations Environment

Everything seems to work in the local notebook

In [ ]:
from kaggle_environments import evaluate, make, utils

env = make("hungry_geese", debug=True)
env.run([
    "main.py", 
    "main.py", 
    "main.py", 
    "main.py"
])
env.render(mode="ipython", width=500, height=450)

# Submission

Multi-file submissions are a new Kaggle feature since Halite.
> Today we released support for multi-file agents. To upload an agent with multiple files, submit your work as a .tar.gz archive (the name must end in .tar.gz) with a python file at the top level called main.py that conforms to the normal agent submission requirements. The maximum upload size of 100MB applies to the full archive. For the initial release of this feature we only compile the main.py file so to import other python files you'll either have to compile/exec or importlib the relevant files. We're hoping to improve this in a future release and will update this topic as appropriate.
> - https://www.kaggle.com/c/halite/discussion/177686

FIXED: This seems to be working now in the Kaggle Submissions Environment!

In [ ]:
!GZIP=-9 tar cfz submission.tar.gz *.py *.so

Quick test to ensure we have all the correct files

In [ ]:
!mkdir test
!cd test; tar xvf ../submission.tar.gz; python3 main.py
!rm -rf test